In [89]:
import heapq
import os
import random
import time

import networkx as nx
import numpy as np
import math
import csv
from sys import maxsize


In [90]:
# def gen_prufer(degree_constrained, n):
#     result = []
#     candidates = list(range(n))
#     while len(result) < n:
#         # Lấy một số ngẫu nhiên từ danh sách candidates
#         x = random.choice(candidates)
#         # Nếu số này chưa xuất hiện quá m lần trong danh sách kết quả, thêm số này vào danh sách kết quả
#         if result.count(x) < degree_constrained - 1:
#             result.append(x)
#         # Nếu số này đã xuất hiện quá m lần, loại bỏ số này khỏi danh sách candidates
#         if result.count(x) == degree_constrained - 1:
#             candidates.remove(x)
#     return result

In [91]:
def gen_prufer(degree_constrained, n):
    # Tạo danh sách ban đầu
    initial_list = list(range(n))
    # Trộn ngẫu nhiên danh sách ban đầu
    random.shuffle(initial_list)
    # Tạo danh sách mới bằng cách lấy (degree_constrained-1) phần tử đầu tiên
    prufer_sequence = initial_list[:degree_constrained - 1]
    # Nếu danh sách mới chưa đủ n-2 phần tử thì lặp lại
    while len(prufer_sequence) < n - 2:
        # Trộn ngẫu nhiên lại danh sách ban đầu
        random.shuffle(initial_list)
        # Lấy phần tử đầu tiên của danh sách ban đầu nếu không nằm trong danh sách mới quá degree_constrained
        for item in initial_list:
            if prufer_sequence.count(item) < degree_constrained:
                prufer_sequence.append(item)
                break
    # Trả về danh sách mới
    return prufer_sequence

In [92]:
# Đọc dữ liệu từ tệp CSV và lưu trữ nó trong danh sách
def get_distance_table(file_path):
    records = []
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader) # Skip header row
        for row in reader:
            records.append(row)

    # Tạo từ điển để lưu trữ khoảng cách giữa các điểm
    distances_table = {}

    # Tính toán khoảng cách giữa các điểm và lưu trữ kết quả vào từ điển
    for i in range(len(records)):
        for j in range(i + 1, len(records)):
            point1 = (float(records[i][1]), float(records[i][2]))
            point2 = (float(records[j][1]), float(records[j][2]))
            distance = math.dist(point1, point2)
            key = (int(records[i][0]), int(records[j][0]))
            distances_table[key] = distance

    # In ra từ điển khoảng cách
    return distances_table


In [93]:
def get_distance(edge, distances_table):
    if edge in distances_table:
        return distances_table[edge]
    if edge[::-1] in distances_table:
        return distances_table[edge[::-1]]

In [94]:
def compute_degrees(edges):
    degrees = {}
    for edge in edges:
        for vertex in edge:
            if vertex not in degrees:
                degrees[vertex] = 0
            degrees[vertex] += 1
    return degrees


In [95]:
def calculate_fitness(prufer_sequence, degree_constrained, distances_table):
    tree = nx.from_prufer_sequence(prufer_sequence)
    edges = tree.edges
    degrees = list(compute_degrees(edges).values())
    # print(degrees)

    # Check if the degrees more than target degrees
    if any(x > degree_constrained for x in degrees):
        return 9999999

    cost = 0
    for edge in edges:
        cost = cost + get_distance(edge, distances_table)
    return cost

In [96]:
def crossover(parent1, parent2, crossover_rate=1):
    offspring1 = parent1[:]
    offspring2 = parent2[:]
    # print(parent1, parent2)
    if random.random() < crossover_rate:
        crossover_point = random.randint(1, len(parent1) - 2)
        # print("c point: ", crossover_point)
        offspring1[crossover_point:], offspring2[crossover_point:] = offspring2[crossover_point:], offspring1[
                                                                                                   crossover_point:]
    return offspring1, offspring2


In [97]:
def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(individual)), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual


def mutate_pop(population, mutation_rate):
    new_pop = []
    for individual in population:
        if random.random() < mutation_rate:
            idx1, idx2 = random.sample(range(len(individual)), 2)
            individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
            new_pop.append(individual)
    return new_pop

In [98]:
# def fix_off(offspring, degree_constrained):
#     counts = {}
#     used = set()
#     new_numbers = []
#
#     for num in offspring:
#         counts[num] = counts.get(num, 0) + 1
#         if counts[num] > degree_constrained:
#             for i in range(0, len(offspring)):
#                 if i not in used:
#                     new_numbers.append(i)
#                     used.add(i)
#                     break
#         else:
#             new_numbers.append(num)
#
#     return new_numbers

In [99]:
def sort_by_value(dict):
    return sorted(dict.items(), key=lambda x: x[1])


def repair(prufer_sequence, degree_constrained):
    set_ver = set(range(len(prufer_sequence) + 2))
    count = {}
    for ver in set_ver:
        count[ver] = prufer_sequence.count(ver)
    count = dict(sort_by_value(count))

    for i in range(len(prufer_sequence)):
        if count[prufer_sequence[i]] > degree_constrained:
            j = next(iter(count))
            count.update([(prufer_sequence[i], count[prufer_sequence[i]] - 1), (j, count[j] + 1)])
            prufer_sequence[i] = j
            count = dict(sort_by_value(count))
    return prufer_sequence

In [100]:
def get_new_pop(population, population_size, degree_constrained, distances_table):
    fitness_values = [calculate_fitness(individual, degree_constrained, distances_table) for
                      individual in population]
    sorted_fitness = sorted(fitness_values)
    index_dict = {val: idx for idx, val in enumerate(fitness_values)}
    # Lấy danh sách index của population đã được sắp xếp theo fitness
    pop_index = [index_dict[val] for val in sorted_fitness]
    # Lấy index của [population_size] phần tử đầu (có cost nhỏ nhất)
    new_pop_index = pop_index[:population_size]
    new_pop = [population[i] for i in new_pop_index]
    return new_pop

In [101]:
def run_ga(n, degree_constrained, distances_table, population_size=50, crossover_rate=0.8, mutation_rate=0.1,
           max_generations=50):
    population = [gen_prufer(degree_constrained, n) for _ in range(population_size)]
    # print(population)
    # print(population)
    for generation in range(max_generations):
        # print(population)
        # Tính một mảng fitness value của population
        fitness_values = [calculate_fitness(prufer_sequence, degree_constrained, distances_table) for
                          prufer_sequence in population]
        # print(fitness_values)
        new_population = []
        while len(new_population) < population_size:
            i, j = np.random.choice(range(population_size), size=2, replace=False,
                                    p=np.array(fitness_values) / sum(fitness_values))
            parent1 = population[i]
            parent2 = population[j]
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.append(offspring1)
            new_population.append(offspring2)
        for i in range(len(new_population)):
            mutate(new_population[i])
            repair(new_population[i], degree_constrained)
        population = population + new_population
        population = get_new_pop(population, population_size, degree_constrained, distances_table)

    return population

In [102]:
# degree_constrained = 2
# n = 5
# distances_table = get_distance_table("data/5_wi29.csv")
#
# population = run_ga(n, degree_constrained, distances_table)
# print(population[0])
# print(calculate_fitness(population[0], degree_constrained, distances_table))
# # fitness_values = [calculate_fitness(prufer_sequence, degree_constrained, distances_table) for prufer_sequence in
# #                           last_generation]
# # print(fitness_values)

In [103]:
# G = nx.from_prufer_sequence(population[0])
# print(G.edges())

In [104]:
# degree_constrained = 2
# n = 7
# distances_table = get_distance_table("data/7_wi29.csv")
#
# population = run_ga(n, degree_constrained, distances_table)
# print(population[0])
# print(calculate_fitness(population[0], degree_constrained, distances_table))
#
# G = nx.from_prufer_sequence(population[0])
# print(G.edges())

In [105]:
import csv
def write_result(results, filename):
    path_result = "result"
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['STT', 'Best Cost', 'Best Solution', 'Best Tree', 'Time'])

        # Ghi từng dòng kết quả
        for result in results:
            writer.writerow(result)

In [106]:
degree_constrained = 3
n = 15
path = "data/15_nodes"
list_file = os.listdir(path)
print(list_file)
for file in list_file:
    print(file)
    results = []
    path_to_data = os.path.join(path, file)
    dis_tab = get_distance_table(path_to_data)
    path_to_result = path_to_data.replace("data", "result")
    path_to_result = path_to_result.replace("\\", "/")
    # print(path_to_result)
    for i in range(10):
        print("Loop: ", i+1, "...")
        start_time = time.time()
        population = run_ga(n, degree_constrained, dis_tab)
        best_solution = population[0]
        best_cost = calculate_fitness(best_solution, n, dis_tab)
        best_tree = nx.from_prufer_sequence(population[0]).edges()
        end_time = time.time()
        elapsed_time = end_time - start_time
        row = (i+1, best_solution, best_cost, best_tree, elapsed_time)
        results.append(row)
    print("Result 10 times", results)

    # print(path_to_result)
    write_result(results, path_to_result)

['10_15_fi10k.csv', '11_15_fi10k.csv', '12_15_fi10k.csv', '13_15_fi10k.csv', '14_15_fi10k.csv', '15_15_fi10k.csv', '16_15_fi10k.csv', '17_15_fi10k.csv', '18_15_fi10k.csv', '19_15_fi10k.csv', '1_15_fi10k.csv', '20_15_fi10k.csv', '21_15_fi10k.csv', '22_15_fi10k.csv', '23_15_fi10k.csv', '24_15_fi10k.csv', '25_15_fi10k.csv', '26_15_fi10k.csv', '27_15_fi10k.csv', '28_15_fi10k.csv', '29_15_fi10k.csv', '2_15_fi10k.csv', '30_15_fi10k.csv', '3_15_fi10k.csv', '4_15_fi10k.csv', '5_15_fi10k.csv', '6_15_fi10k.csv', '7_15_fi10k.csv', '8_15_fi10k.csv', '9_15_fi10k.csv']
10_15_fi10k.csv
Loop:  1 ...
Loop:  2 ...
Loop:  3 ...
Loop:  4 ...
Loop:  5 ...
Loop:  6 ...
Loop:  7 ...
Loop:  8 ...
Loop:  9 ...
Loop:  10 ...
Result 10 times [(1, [4, 13, 2, 0, 13, 12, 5, 6, 7, 14, 14, 11, 9], 25359.483930457693, EdgeView([(0, 2), (0, 13), (1, 4), (2, 4), (3, 13), (5, 10), (5, 6), (6, 7), (7, 14), (8, 12), (9, 11), (9, 14), (11, 13), (12, 14)]), 2.199742317199707), (2, [5, 1, 5, 8, 10, 2, 12, 9, 9, 13, 3, 4, 0], 